In [117]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split, Subset
from torchvision import transforms, datasets
from tqdm import tqdm
from pathlib import Path

In [118]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Make those seeds so u don't have random result iykwim
torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)


# Prepping out that data

Set Base Varibles

In [119]:
BATCH_SIZE =32
NUM_WORKERS =0
IMG_SIZE = 224 #Use 256 if ur gonna crop it more laterr
BASE_DIR = "Multi-class Weather Dataset"
CLOUDY_DIR = "Multi-class Weather Dataset/Cloudy"
RAIN_DIR = "Multi-class Weather Dataset/Rain"
SHINE_DIR = "Multi-class Weather Dataset/Shine"
SUNRISE_DIR = "Multi-class Weather Dataset/Sunrise"


In [120]:
# Basic
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

#### Transformations

In [121]:
train_transformers = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.5),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    normalize
])

In [122]:
val_transformers = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    normalize
])

In [123]:
#### Ok so the problem is:
# 1.  I have my files like master folder and then each class so i can use the datasets.ImageFolder thingy which makes having the dataset really ez so i don't have to customly define one
# 2. BUT i need to apply different transformations on the different splits of the dataset but with the datasets.Image Set up i can only apply one transformation normally

## So to fix this issue  I use No transformations at first and then determine the different sizes of the different splits  and THEN i apply the different transformations to each subset that then pass through the the apply_transforms function class thy that then gets transformed and then gets put into these dataloaders

 ### Don't use this data loading strategy as a model for your other ones.

In [124]:
FULL_dataset = datasets.ImageFolder(BASE_DIR, transform=None)

In [125]:
train_size = int(len(FULL_dataset) * 0.8)
val_size = int(len(FULL_dataset) * 0.1)
test_size = len(FULL_dataset) - train_size-val_size

In [126]:
train_subset, val_subset, test_subset = random_split(
    FULL_dataset, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

In [127]:
def apply_transformations(subset, transforms_top):
    class DatasetwitTransformations(Dataset):
        def __init__(self, subset, transforms_top):

            super().__init__()

            self.subset = subset
            self.transforms_top = transforms_top

            self.classes = subset.dataset.classes
            self.class_to_idx = subset.dataset.class_to_idx
        def __getitem__(self, idx):

            img, label = self.subset[idx]
            if self.transforms_top is not None:
                img = self.transforms_top(img)
            return img, label
        def __len__(self):
            return len(self.subset)
    return DatasetwitTransformations(subset, transforms_top)

In [128]:
train_dataset_transformed = apply_transformations(train_subset, train_transformers)
valid_dataset_transformed = apply_transformations(val_subset, val_transformers)
test_dataset_transformed = apply_transformations(test_subset, val_transformers)

In [129]:
train_loader = DataLoader(train_dataset_transformed, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset_transformed, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_dataset_transformed, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=False, pin_memory=True)

In [130]:
NUM_CLASSES = len(FULL_dataset.classes)
CLASS_NAMES = FULL_dataset.classes

In [131]:
# I needa add more debugging steps like ts

In [132]:
print(f'DATALOADING STEPS RESULTS.....')
print(f"\nNumber of classes: {NUM_CLASSES}")
print(f"Class Names: {CLASS_NAMES}")
print(f"\nNumber of images in Training dataset: {len(train_dataset_transformed)}")
print(f"Number of images in Validation dataset:{len(valid_dataset_transformed)}")
print(f"Number of images in Test dataset:{len(test_dataset_transformed)}")
print("\nData loading strat was sucessful")

DATALOADING STEPS RESULTS.....

Number of classes: 4
Class Names: ['Cloudy', 'Rain', 'Shine', 'Sunrise']

Number of images in Training dataset: 900
Number of images in Validation dataset:112
Number of images in Test dataset:113

Data loading strat was sucessful


## DEFINING THE CNN

In [133]:
### Ok making the conv block so ion have to repeat it (I AM NEVER DOING TS AGAIN. IMA JUST USE TRANSFERLEARNING AFTER THIS PROJECT LOL. WHY DID I THINK I NEEDED TO KNOW ALL THISE (althought i learned a lot from this tho)

In [134]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernal_size=3, pool=True):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernal_size, padding=1)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        if self.pool is not None:
            x = self.pool(x)
        return x



In [135]:
class WeatherClassifierCNN(nn.Module):
    def __init__(self, num_classes, input_channels=3):
        super(WeatherClassifierCNN, self).__init__()
        self.features = nn.Sequential(
            ConvBlock(input_channels, 64, pool=True),
            ConvBlock(64, 128, pool=True),
            ConvBlock(128, 256, pool=True),
            ConvBlock(256, 512, pool=True),
        )

        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        #classfier part

        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.gap(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x



In [136]:
model = WeatherClassifierCNN(num_classes=NUM_CLASSES)
model = model.to(DEVICE)

# TRAINING PHASE!!!

Traning setup

In [137]:
NUM_EPOCH = 10
LEARNING_RATE = 0.001
best_val_loss = float('inf')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [138]:
SAVE_DIR = "checkpoints"
Path(SAVE_DIR).mkdir(parents=True, exist_ok=True)

In [139]:

schuduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer=optimizer, mode='min', factor=0.5, patience=5
)

Training Loop

In [140]:
print("TRAINING IS STARTING NOW!!")

for epoch in range(NUM_EPOCH):
    print(F"\n Epoch({epoch+1}/{NUM_EPOCH})")

    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    progress_bar = tqdm(train_loader, desc="Training", leave=False)

    for inputs, labels in progress_bar:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()

        # forward pas
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # the backward pass and step

        loss.backward()
        optimizer.step()

        #Getting results

        train_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted ==  labels).sum().item()

        #progress bar update
        progress_bar.set_postfix({'loss': loss.item(), 'acc': 100 * train_correct / train_total})

    epoch_train_loss = train_loss / train_total
    epoch_train_acc = 100 * train_correct / train_total

    model.eval()

    val_loss = 0.0
    val_correct = 0
    val_total = 0

    progress_bar = tqdm(valid_loader, desc="Validation", leave=False)

    with torch.no_grad():
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

            #forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

            progress_bar.set_postfix({'loss' : loss.item(), 'acc': 100 * val_correct / val_total})

        epoch_val_loss = val_loss / val_total
        epoh_val_acc = 100 * val_correct / val_total

        schuduler.step(epoch_val_loss)


        print(f'\nResults......')
        print(f"\nTrainloss is: {epoch_train_loss:.4} --While Training Accuracy is: {epoch_train_acc:.4}")
        print(f"\n Validation loss is: {epoch_val_loss:.4} ---While Validation accuracy is {epoh_val_acc:.4}")

        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            best_epoch = epoch+1
            checkpoint = {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'accuracy': best_val_loss,
            }
            torch.save(checkpoint, Path(SAVE_DIR) / 'best_model.pth')
            print(f"\nBest Model was Saved in {SAVE_DIR} with the Validation loss of {best_val_loss:.4}")
        else:
            print("\nModel Failed to Improve from {best_val_loss:.4} at Epoch {best_epoch}")


TRAINING IS STARTING NOW!!

 Epoch(1/10)


KeyboardInterrupt: 

Post Project Reflections

In [88]:
# Ok so i need to learn more about the Dataset and Dataloader classes in pytorch
# Next project probably litterly the same exact thing except for i just have a different dataset with a different file structure and a use the custom dataset class. I needa rewatch some yt vids I think i might just do a course for now on. AND I JUST DISCOVERED MY GPU ISN'T HAVE BAD TO CODE WITH LOLOLOL its taking about  2 minutes per epoches so i might as well just change it to five epochs total and
#
#
# i don't even know what to type here so i ain't gonna type NOTHING lool
#nahh gang ts crazy damm ts crazy ick

RESULTS AND DATA ANAL

Check points!!!!!

In [143]:
checkpoint =  torch.load(Path(SAVE_DIR) / 'best_model.pth', map_location=DEVICE)
model.load_state_dict(checkpoint['model_state_dict'])
print(f'\n MODEL IS LOADED ---Epoch: {checkpoint['epoch']}, Accuracy : {checkpoint['accuracy']:.4}')
print(" Model is LOADED!!!!! Have fun with validation ]")


 MODEL IS LOADED ---Epoch: 5, Accuracy : 0.3411
 Model is LOADED!!!!! Have fun with validation ]


In [ ]:
# Ok we need to make a gradio version of this app